In [4]:
import numpy as np
import random
from FPU import cmul, cadd
from utils import conj, read_binary, binary_to_fp16, store_binary, generate_fp16, fp16_to_binary, int_to_binary, binary_to_int
from ed import ed

# Basic functions

In [ ]:
def delayseq(data, lag: int, mode: str = "zero"):
    """
    Apply delay (positive lag) or advance (negative lag) to a sequence.

    Supports both:
    - Real sequences (list of np.float16)
    - Complex sequences (list of [real, imag] pairs)

    Output length is always equal to the input length.

    Args:
        data: Input sequence (real or complex).
        lag:  Number of samples to delay (positive) or advance (negative).
        mode:
            "zero" → fill shifted positions with zeros (default).
            "wrap" → perform circular (cyclic) shift.

    Returns:
        Delayed (or advanced) sequence of the same length.
    """
    if not data:
        return []

    # Detect if the input is complex
    is_complex = isinstance(data[0], (list, tuple)) and len(data[0]) == 2
    n = len(data)

    # Circular shift mode
    if mode == "wrap":
        k = lag % n
        return data[-k:] + data[:-k] if k != 0 else data.copy()

    if mode != "zero":
        raise ValueError('mode must be "zero" or "wrap"')

    # Zero-fill mode
    zero_val = [np.float16(0.0), np.float16(0.0)] if is_complex else np.float16(0.0)

    # Positive lag → delay to the right (zeros at the front)
    if lag > 0:
        fill = [zero_val] * min(lag, n)
        tail = data[: max(0, n - lag)]
        return fill + tail

    # No lag
    if lag == 0:
        return data.copy()

    # Negative lag → advance to the left (zeros at the end)
    lag_abs = min(-lag, n)
    head = data[lag_abs:]
    fill = [zero_val] * lag_abs
    return head + fill

# Custom Kernel

## CP Detect